### 📦 Install Antigranular

This command installs the [Antigranular PyPI Package](https://pypi.org/project/antigranular/) on the local enviroment.


In [ ]:
# Install the Antigranular package
!pip install antigranular &> /dev/null

### ✍ Login to the Enclave

Head over to the [Competition](https://www.antigranular.com/competitions/global-economic-forecast-hackathon-with-texas-a-m-aggie-data-science) to find your `<user_id>`, `<user_secret>` and the competition's name and copy that command here.

![img](https://docs.antigranular.com/shots/comp_cell.png)

In [ ]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Fraud Detection Hackathon with Privacy Village #DPD25FEST")

Dataset "Transaction Fraud Hackathon Dataset" loaded to the kernel as transaction_fraud_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
test_x                         DataFrame      
train_x                        PrivateDataFrame
train_y                        PrivateDataFrame

Connected to Antigranular server session id: 9995f04b-4c35-455d-8e6c-f4835fd7c9aa, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [ ]:
session.privacy_odometer()

{'total_epsilon_used': 1.3010015549999998,
 'total_delta_used': 0.0,
 'library_costs': {'op_pandas': {'total_delta': 0.0,
   'total_epsilon': 5.5e-08,
   'total_requests': 10},
  'op_dpl': {'total_delta': 0.0,
   'total_epsilon': 1.3010014999999997,
   'total_requests': 9}},
 'dataset_costs': {'Transaction Fraud Hackathon Dataset': {'delta': 0.0,
   'eps': 1.3010015549999998}}}

## 🤖 2. A simple guide of AG

You can now simply use ``%%ag`` to run code on an enclave! You can always head over to our [Docs](https://docs.antigranular.com/) to learn more about AG, but for now, we can define train and test variables as follows.

In [ ]:
%%ag
x_train = transaction_fraud_hackathon_dataset["train_x"]
y_train = transaction_fraud_hackathon_dataset["train_y"]
x_test = transaction_fraud_hackathon_dataset["test_x"]

### 🕵️‍♂️ Exploring data

Exploring data in Antigranular involves spending your epsilon budget, be mindful of your usage but remember that the less epsilon you use, the less accurate your results will get!

<img src="https://static.vecteezy.com/system/resources/thumbnails/016/589/195/small/light-bulb-8bit-pixel-png.png" alt="Lightbulb" width="50px" />

#### 🌀 Want to learn more about the data?

For this competition, we are using multiple fraud prediction indicators. Although we are not able to share the exact indicators, you can think of them as factors such as "Was this transaction an amount that is usually spent by this user?" or "How many transactions were performed this day?", which are then normalized.



In [ ]:
%%ag
x_train.info()

+----+----------+-------------+---------------+---------+----------+
|    | Column   | numerical   | categorical   | dtype   | bounds   |
|----+----------+-------------+---------------+---------+----------|
|  0 | a0       | True        | False         | float64 | (0, 1)   |
|  1 | a1       | True        | False         | float64 | (0, 1)   |
|  2 | a2       | True        | False         | float64 | (0, 1)   |
|  3 | a3       | True        | False         | float64 | (0, 1)   |
|  4 | a4       | True        | False         | float64 | (0, 1)   |
|  5 | a5       | True        | False         | float64 | (0, 1)   |
|  6 | a6       | True        | False         | float64 | (0, 1)   |
|  7 | a7       | True        | False         | float64 | (0, 1)   |
|  8 | a8       | True        | False         | float64 | (0, 1)   |
+----+----------+-------------+---------------+---------+----------+



For our `Y` target, we are trying to predict if the sample constitutes fraud, being this a binary classification problem.


```
Fraud:
    1. 'Not fraud': 0,
    2. 'Fraud': 1,
```




In [ ]:
%%ag
y_train.info()

+----+----------+-------------+---------------+---------+----------+
|    | Column   | numerical   | categorical   | dtype   | bounds   |
|----+----------+-------------+---------------+---------+----------|
|  0 | fraud    | True        | False         | float64 | (0, 1)   |
+----+----------+-------------+---------------+---------+----------+



## 🧠 4. The Solution

In this section we evaluate an editorial solution in AG using TensorFlow!

In [ ]:
%%ag
import tensorflow as tf
from op_pandas import standard_scaler, PrivateDataFrame
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv1D
from op_tensorflow import PrivateKerasModel, PrivateDataLoader


# Keras model
seqM = Sequential([
    Dense(512, activation='relu', input_shape=(9,)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Create DP keras model
dp_model = PrivateKerasModel(model=seqM, l2_norm_clip=1.0, noise_multiplier=1.0)

# Use a standard (non-DP) optimizer directly from keras.
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# PrivateKerasModel uses similar API as standard Keras
dp_model.compile(
	optimizer = optimizer,
	loss = 'binary_crossentropy',
	metrics=["accuracy"]
)

In [ ]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train, label_df=y_train, batch_size=200)

In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=25, target_delta=1)

Epoch 1/25

175/175 - 23s - loss: 0.6624 - accuracy: 0.6231 - 23s/epoch - 134ms/step

Epoch 2/25

175/175 - 16s - loss: 0.6265 - accuracy: 0.6542 - 16s/epoch - 89ms/step

Epoch 3/25

175/175 - 15s - loss: 0.6179 - accuracy: 0.6590 - 15s/epoch - 86ms/step

Epoch 4/25

175/175 - 15s - loss: 0.6082 - accuracy: 0.6711 - 15s/epoch - 86ms/step

Epoch 5/25

175/175 - 15s - loss: 0.6096 - accuracy: 0.6670 - 15s/epoch - 86ms/step

Epoch 6/25

175/175 - 16s - loss: 0.6053 - accuracy: 0.6728 - 16s/epoch - 90ms/step

Epoch 7/25

175/175 - 15s - loss: 0.6016 - accuracy: 0.6765 - 15s/epoch - 87ms/step

Epoch 8/25

175/175 - 15s - loss: 0.5976 - accuracy: 0.6826 - 15s/epoch - 86ms/step

Epoch 9/25

175/175 - 15s - loss: 0.5967 - accuracy: 0.6789 - 15s/epoch - 86ms/step

Epoch 10/25

175/175 - 15s - loss: 0.5975 - accuracy: 0.6769 - 15s/epoch - 86ms/step

Epoch 11/25

175/175 - 15s - loss: 0.5986 - accuracy: 0.6765 - 15s/epoch - 86ms/step

Epoch 12/25

175/175 - 15s - loss: 0.5936 - accuracy: 0.6824 -

In [ ]:
%%ag
y_pred = dp_model.predict(PrivateDataFrame(x_test), label_columns=["output"])

469/469 [==============================] - 4s 8ms/step



In [ ]:
%%ag
# Note that the predictions are a float scalar, so we scale it!
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))

In [ ]:
%%ag
result = submit_predictions(y_pred)

score: {'leaderboard': 0.68611576595777, 'logs': {'BIN_ACC': 0.69262077373277, 'LIN_EPS': -0.006505007774999999}}



In [ ]:
session.terminate_session()

{'status': 'ok'}